In [5]:
# pip uninstall tensorflow tensorflow-gpu keras transformers
# pip install tensorflow-gpu==2.10.1 keras==2.10.0 protobuf==3.19.6 transformers==4.26.1

# import tensorflow as tf
# print(tf.__version__)
# print("GPUs:", tf.config.list_physical_devices('GPU'))

import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)


Number of GPU:  1
GPU Name:  NVIDIA GeForce RTX 3060 Laptop GPU
Using device: cuda


model inference

ngrok/google cloud/local

In [ ]:
# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-19_21-00-49.mp3"
# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-14_12-41-34.mp3"
# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-14_12-46-07.mp3"
# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-19_20-46-14.mp3"

# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-55-11.ogg" # ok lepah demo gi kighi, gi straight ja
# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-55-06.ogg" # dok bakpo ni
# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-55-02.ogg" # macei kite tengok oghe tu buat kijo
# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-54-58.ogg" # nikoh memang banyak kijo pon
# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-54-52.ogg" # mung nok tubiq mano


In [4]:
import requests

#kedah
# url = "http://localhost:8080/transcribe"
url = "https://1681-175-141-188-34.ngrok-free.app/transcribe/"
files = {'file': open(r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-14_12-41-34.mp3", 'rb')}
# files = {'file': open(r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-19_20-46-14.mp3", 'rb')}
# files = {'file': open(r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-14_12-41-34.mp3", 'rb')}
# files = {'file': open(r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-14_12-41-34.mp3", 'rb')}
response = requests.post(url, files=files)
print(response.status_code)
print(response.json())

#kelantan
# url = "https://lingomalay-api-1092407226826.asia-southeast1.run.app/transcribe/"
# url = "http://localhost:8081/transcribe"
url = "https://c5b2-175-141-188-34.ngrok-free.app/transcribe/"
files = {'file': open(r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-55-02.ogg", 'rb')}
response = requests.post(url, files=files)
print(response.status_code)
print(response.json())

200
{'transcription': ' Tang pasaipa  嗯 嗯 嗯 嗯 嗯 嗯 嗯 嗯 嗯 嗯 嗯 嗯 嗯 '}
200
{'transcription': ' Macei kite tengok oghe tu buat kijo                    '}


checking json data

In [2]:
from datasets import load_dataset, Audio
dataset2 = load_dataset(
    "json",
    data_files=r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\#Main(clean)\##Transcripts2.jsonl",
    split="train"
)
dataset2 = dataset2.cast_column("audio", Audio(sampling_rate=16000))
print(dataset2[50])

{'text': 'Haa, apa yang pancit? Aku tak paham hang sembang pasai apa. Ooo, tayar motor. Kedah mana ade dok tayaq-tayaq motoq-motoq, pandai-pandai ja hang ni. Sat, aku tukaq tayar.', 'language': 'ms', 'dialect': 'kedah', 'audio': {'path': 'D:\\LingoMalay\\Models_Transcribe\\Dataset\\Kedah\\#Main(clean)\\#7 kartun3_3.wav', 'array': array([ 8.97216797e-02,  1.21917725e-01,  1.06872559e-01, ...,
        0.00000000e+00, -3.05175781e-05, -3.05175781e-05]), 'sampling_rate': 16000}}


models testing

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
import torchaudio
import os
import random

# Load fine-tuned model
model = WhisperForConditionalGeneration.from_pretrained(r"D:\LingoMalay\Models_Transcribe\API\Kedah\app\model")
processor = WhisperProcessor.from_pretrained(r"D:\LingoMalay\Models_Transcribe\API\Kedah\app\model")
# kedah v4,200 best
# v5,200 not bad, but v4 better

# kelate v2, 1000 best
# v1v2, 50 -> v1v2v2 100 ->
# v3, 150

model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load audio (ensure it's 16kHz mono)
# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-19_21-00-49.mp3"
# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-14_12-41-34.mp3"
# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-14_12-46-07.mp3"
# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-19_20-46-14.mp3"

# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-55-11.ogg" # ok lepah demo gi kighi, gi straight ja
# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-55-06.ogg" # dok bakpo ni
# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-55-02.ogg"  # macei kite tengok oghe tu buat kijo
# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-54-58.ogg" # nikoh memang banyak kijo pon
# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-54-52.ogg" # mung nok tubiq mano
# audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\#Main(clean)\#8 clip_56.wav"

folder_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\#Main(clean)"
# folder_path = r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\#Main(clean)"
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
random_file = random.choice(files)
audio_path = os.path.join(folder_path, random_file)
print(audio_path)


waveform, sr = torchaudio.load(audio_path)
if sr != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
    waveform = resampler(waveform)
waveform = waveform.squeeze().numpy()  # Convert to 1D numpy array

inputs = processor.feature_extractor(waveform, sampling_rate=16000, return_tensors="pt")
input_features = inputs.input_features.to(device)
# attention_mask = torch.ones(input_features.shape, dtype=torch.long).to(device)  # force full attention

# slow inference, not recommended
# # prompt = "<|startoftranscript|><|ms|><|kedah|><|transcribe|> "
# prompt_tokens = processor.tokenizer.convert_tokens_to_ids([
#     # "<|startoftranscript|>", "<|ms|>", "<|kedah|>", "<|transcribe|>",
# ])
# forced_decoder_ids = list(enumerate(prompt_tokens))

# model.generation_config.forced_decoder_ids = None  # Disable forced decoder IDs for now

# forced_decoder_ids = processor.get_decoder_prompt_ids(language="ms", task="transcribe")
# forced_decoder_ids += [(len(forced_decoder_ids), processor.tokenizer.convert_tokens_to_ids("<|kedah|>"))]

with torch.no_grad():
    predicted_ids = model.generate(
        input_features.to(device),
        # attention_mask=attention_mask,
        # forced_decoder_ids=forced_decoder_ids,
        # max_length=448,
        max_new_tokens=32,
        # eos_token_id=processor.tokenizer.eos_token_id,  # <--- this is crucial now
        # early_stopping=True,
        # num_beams=1,
        # repetition_penalty=1.2,
        # no_repeat_ngram_size=3,
        # do_sample=False
    )

transcription = processor.tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)[0]
print(transcription)

# Greedy decode + cut at eos manually
# output_ids = predicted_ids[0].tolist()
# if processor.tokenizer.eos_token_id in output_ids:
#     output_ids = output_ids[:output_ids.index(processor.tokenizer.eos_token_id)]
# text = processor.tokenizer.decode(output_ids, skip_special_tokens=True)
# print("Transcription:", text)

D:\LingoMalay\Models_Transcribe\Dataset\Kedah\#Main(clean)\#5 (L) Ustaz Lukmanul Hakim ｜ Perangai Tak Senonoh ! [pMrYZKV4zn8]_21.wav
 Baik kita tengok die bagi lagi kepada kita apa nama, contoh lagi, dan membawak fitnah besaq dan peghang


models performance comparison

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
import torchaudio
from datasets import load_dataset, Dataset
from evaluate import load
import librosa
import re

# Load models
models = {
    "mesolotica": WhisperForConditionalGeneration.from_pretrained("mesolitica/malaysian-whisper-small-v3"),
    # "openai": WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3"),
    # "me": WhisperForConditionalGeneration.from_pretrained(r"D:\LingoMalay\Models_Transcribe\API\Kelantan\app\model"),
    # "me": WhisperForConditionalGeneration.from_pretrained(r"D:\LingoMalay\Models_Transcribe\Model\Kedah\whisper-kedahv3\checkpoint-1000")
}
processors = {
    name: WhisperProcessor.from_pretrained(model_name)
    for name, model_name in {
        "mesolotica": "mesolitica/malaysian-whisper-small-v3",
        # "openai": "openai/whisper-large-v3"
        # "me": r"D:\LingoMalay\Models_Transcribe\API\Kelantan\app\model"
        # "me": r"D:\LingoMalay\Models_Transcribe\Model"
    }.items()
}

# Example: list of dictionaries
data = [
    {"audio": {"path": r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-19_21-00-49.mp3"}, "sentence": "awat tahabaq mai"},
    {"audio": {"path": r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-19_20-46-14.mp3"}, "sentence": "tak celuih laa"},
    {"audio": {"path": r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-07-04_17-23-16.ogg"}, "sentence": "hang pasaipa"},
    {"audio": {"path": r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-07-04_17-48-14.ogg"}, "sentence": "depa nak pi mana tu"},
    {"audio": {"path": r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-07-04_17-15-56.ogg"}, "sentence": "loqlaq la hang ni"},
]

# data = [
#     {"audio": {"path": r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-55-11.ogg"}, "sentence": "ok lepah demo gi kighi, gi straight ja"},
#     {"audio": {"path": r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-55-06.ogg"}, "sentence": "dok bakpo ni"},
#     {"audio": {"path": r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-55-02.ogg"}, "sentence": "macei kite tengok oghe tu buat kijo"},
#     {"audio": {"path": r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-54-58.ogg"}, "sentence": "nikoh memang banyak kijo pon"},
#     {"audio": {"path": r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-54-52.ogg"}, "sentence": "mung nok tubik mano"}
# ]

ds = Dataset.from_list(data)

# cer/wer
wer = load("wer")
cer = load("cer")

def normalize(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    return text.strip()

def transcribe(audio_path, model, processor):
    # Load audio with librosa at 22050 Hz (default for librosa)
    waveform, sample_rate = librosa.load(audio_path, sr=None)  # You can replace 22050 with None for the default sample rate

    # Resample to 16000 Hz (required by Whisper)
    if sample_rate != 16000:
        waveform = librosa.resample(waveform, orig_sr=sample_rate, target_sr=16000)
    
    # Process the waveform
    inputs = processor(waveform, sampling_rate=16000, return_tensors="pt")
    
    # Generate transcription
    with torch.no_grad():
        pred_ids = model.generate(**inputs, forced_decoder_ids=processor.get_decoder_prompt_ids(language="ms", task="transcribe"))
    
    # return processor.batch_decode(pred_ids, skip_special_tokens=True)[0]
    output = processor.batch_decode(pred_ids, skip_special_tokens=True)[0]
    clean_output = output.replace("startoftranscript", "").replace("ms", "").replace("transcribe", "").replace("notimestamps", "").strip()
    return clean_output

# Run benchmark
for name in models:
    predictions = []
    references = []
    for example in ds:
        ref = normalize(example['sentence'])
        audio = example['audio']['path']
        pred = normalize(transcribe(audio, models[name], processors[name]))
        references.append(ref)
        predictions.append(pred)
        print(f"Prediction: {pred}")
        print(f"Ground Truth: {ref}")

    Wscore = wer.compute(predictions=predictions, references=references)
    Cscore = cer.compute(predictions=predictions, references=references)
    print(f"WER for {name}: {Wscore:.4f}")
    print(f"CER for {name}: {Cscore:.4f}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Prediction: awak terhabak max
Ground Truth: awat tahabaq mai
Prediction: tak celih lah
Ground Truth: tak celuih laa
Prediction: hang pasai pa
Ground Truth: hang pasaipa
Prediction: depanak pimana tu
Ground Truth: depa nak pi mana tu
Prediction: lok la le hang ni
Ground Truth: loqlaq la hang ni
WER for mesolotica: 0.7647
CER for mesolotica: 0.1795


Codes below are not used

In [ ]:
import soundfile as sf
from transformers import pipeline, WhisperForConditionalGeneration, WhisperProcessor, AutoTokenizer, T5ForConditionalGeneration, WhisperFeatureExtractor

# model_path = 'mesolitica/malaysian-whisper-small-v3'
# model_path = 'mesolitica/Malaysian-whisper-large-v3-turbo-v3'
model_path = r"D:\LingoMalay\Models_Transcribe\Model\Kelantan\whisper-kelantanv2\checkpoint-1000"
# model_path = "openai/whisper-small"

# base_model = 'openai/whisper-small'

# processor = WhisperProcessor.from_pretrained(base_model)
# tokenizer = AutoTokenizer.from_pretrained(base_model)

# Load processor and model from the same local path
processor = WhisperProcessor.from_pretrained(r"D:\LingoMalay\Models_Transcribe\Model")
# processor = WhisperProcessor.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = WhisperForConditionalGeneration.from_pretrained(model_path)

# model.generation_config.forced_decoder_ids = None

# print(processor.feature_extractor)

asr = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=tokenizer,
    feature_extractor=processor.feature_extractor,
    device=0  # set to 0 if using GPU, or -1 for CPU
)

# Run inference
# result = [asr(r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-19_21-00-49.mp3")['text']]
# result = [asr(r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-14_12-41-34.mp3")['text']]
# result = [asr(r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-14_12-46-07.mp3")['text']]
# result = [asr(r"D:\LingoMalay\Models_Transcribe\Dataset\Kedah\audio_2025-04-19_20-46-14.mp3")['text']]

# result = [asr(r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-55-11.ogg")['text']] # mu nak tubiq mano
# result = [asr(r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-55-06.ogg")['text']] # nikoh memang banyak kijo pon
# result = [asr(r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-55-02.ogg")['text']]  # mace kite tengok oghe tu wat kijo
# result = [asr(r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-54-58.ogg")['text']] # dok bakpo ni 
# result = [asr(r"D:\LingoMalay\Models_Transcribe\Dataset\Kelantan\audio_2025-06-18_01-54-52.ogg")['text']] # ok lepah demo gi kighi, gi straight jah

# print(result)


result = ['mung dok tubik ke']
# result = ['macam kita tengok orang tu buat kerja']
# result = ['ghumah aku kat situ je']
# result = ['awak nak pergi mana tu? saya nak ikut boleh?']
# result = ['我想要去哪里？我能跟你吗？']
# result = ['Where did you want to go? Can I follow you?']

tokenizer = AutoTokenizer.from_pretrained('mesolitica/nanot5-base-malaysian-translation-v2.1')
model = T5ForConditionalGeneration.from_pretrained('mesolitica/nanot5-base-malaysian-translation-v2.1')

# # tamil not accurate
# # mandarin quite good

all_special_ids = [0, 1, 2]
prefix = 'terjemah ke Inggeris: '
input_ids = [{'input_ids': tokenizer.encode(f'{prefix}{s}{tokenizer.eos_token}', return_tensors='pt')[
    0]} for s in result]
padded = tokenizer.pad(input_ids, padding='longest')
outputs = model.generate(**padded, max_length = 100)
tokenizer.batch_decode([[i for i in o if i not in all_special_ids] for o in outputs])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[" I don't know what to do"]

In [ ]:
save_path = r"D:\LingoMalay\Models_Transcribe\whisper-kedahv2\checkpoint-800"

processor = WhisperProcessor.from_pretrained("mesolitica/Malaysian-whisper-small-v3")
processor.save_pretrained(save_path)

[]

In [ ]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torchaudio

# === Load Model & Processor ===
model_path = r"D:\LingoMalay\Models_Transcribe\whisper-kedahv4\checkpoint-600"
processor = WhisperProcessor.from_pretrained(model_path)
model = WhisperForConditionalGeneration.from_pretrained(model_path).cuda()
model.eval()

# === Load Audio ===
audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\extra\audio_2025-04-19_21-00-49.mp3"
waveform, sample_rate = torchaudio.load(audio_path)

# === Resample if needed ===
if sample_rate != 16000:
    waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)

# === Prepare input ===
inputs = processor(
    waveform.squeeze().numpy(),
    sampling_rate=16000,
    return_tensors="pt"
)
inputs["input_features"] = inputs["input_features"].cuda()

# === Build decoder prompt ===
prompt = processor.get_decoder_prompt_ids(language="ms", task="transcribe")
# Insert your dialect token after <|ms|>
dialect_token_id = processor.tokenizer.convert_tokens_to_ids("<kedah>")
# Example: [<|startoftranscript|>, <|ms|>] → insert <|kedah|> after <|ms|>
custom_prompt_ids = prompt[:2] + [(1, dialect_token_id)] + prompt[2:]
prompt_ids_tensor = torch.tensor([[token_id for _, token_id in custom_prompt_ids]]).cuda()

model.generation_config.forced_decoder_ids = None

# === Inference ===
with torch.no_grad():
    output = model.generate(
        inputs["input_features"],
        decoder_input_ids=prompt_ids_tensor,
        return_dict_in_generate=True,
        return_timestamps=False,
        output_scores=False,
        max_new_tokens=32,
        do_sample=False,
        num_beams=1,
        repetition_penalty=1.2,
        no_repeat_ngram_size=3
    )

# === Decode ===
transcription = processor.batch_decode(output["sequences"], skip_special_tokens=True)[0]
print("🔹 Transcription:", transcription)

🔹 Transcription:  T J L Ter B S Saooookokok C Mac Mac Mac macam lalalalalarlala


In [ ]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# === 1. Load Model & Processor ===
processor = WhisperProcessor.from_pretrained("mesolitica/malaysian-whisper-small-v3")
model = WhisperForConditionalGeneration.from_pretrained(r"D:\LingoMalay\Models_Transcribe\whisper-kedahv2\checkpoint-800").to("cuda")

# === 2. Prepare Input ===
audio_input = processor.feature_extractor(audio_array, sampling_rate=16000, return_tensors="pt").input_features.to("cuda")

# === 3. Setup Proper Prompt ===
prompt_tokens = processor.tokenizer.convert_tokens_to_ids([
    "<|startoftranscript|>", "<|ms|>", "<|kedah|>", "<|transcribe|>"
])
forced_decoder_ids = list(enumerate(prompt_tokens))  # No <|endoftext|> here

# === 4. Generate Prediction ===
with torch.no_grad():
    predicted_ids = model.generate(
        audio_input,
        forced_decoder_ids=forced_decoder_ids,
        max_new_tokens=128,  # adjust as needed
        eos_token_id=processor.tokenizer.eos_token_id,
        early_stopping=True,
        num_beams=5,  # or 1 if you want greedy
        repetition_penalty=1.2,
        no_repeat_ngram_size=3,
        do_sample=False
    )

# === 5. Decode Output ===
output_text = processor.tokenizer.decode(predicted_ids[0], skip_special_tokens=True)
print(output_text)


OSError: your-model-dir is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torchaudio

# === 1. Load fine-tuned model and processor ===
model = WhisperForConditionalGeneration.from_pretrained(r"D:\LingoMalay\Models_Transcribe\whisper-kedahv2\checkpoint-800")  # path to your fine-tuned model
processor = WhisperProcessor.from_pretrained("mesolitica/malaysian-whisper-small-v3")

model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# === 2. Load and resample audio ===
def load_audio(path):
    audio, sr = torchaudio.load(path)
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(sr, 16000)
        audio = resampler(audio)
    return audio.squeeze().numpy()

audio_path = r"D:\LingoMalay\Models_Transcribe\Dataset\extra\audio_2025-04-19_21-00-49.mp3"
input_audio = load_audio(audio_path)

# === 3. Extract features ===
inputs = processor.feature_extractor(input_audio, sampling_rate=16000, return_tensors="pt")
input_features = inputs.input_features.to(device)

# === 4. Optional: add prompt (to guide dialect)
prompt = "<|kedah|>"
# prompt = "<|startoftranscript|><|ms|><|kedah|><|transcribe|> "
prompt_ids = processor.tokenizer(prompt, return_tensors="pt").input_ids.to(device)

model.generation_config.forced_decoder_ids = None

# === 5. Generate prediction ===
with torch.no_grad():
    generated = model.generate(
        input_features,
        decoder_input_ids=prompt_ids,
        return_timestamps=False,
        return_dict_in_generate=True,
        output_scores=False,
        max_new_tokens=32,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
    )

# === 6. Decode ===
transcription = processor.tokenizer.batch_decode(generated.sequences, skip_special_tokens=True)[0]
print("Transcription:", transcription)


In [ ]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torchaudio

# === Load model and processor ===
model_id = r"D:\LingoMalay\Models_Transcribe\whisper-kedahv2\checkpoint-1000" # or your custom fine-tuned model
processor = WhisperProcessor.from_pretrained(model_id)
model = WhisperForConditionalGeneration.from_pretrained(model_id).cuda()
model.eval()

# === Load audio ===
# audio_path = "D:/LingoMalay/Models_Transcribe/Dataset/extra/audio_2025-04-19_21-00-49.mp3"
# audio_path = "D:/LingoMalay/Models_Transcribe/Dataset/extra/audio_2025-04-14_12-41-34.mp3"
audio_path = "D:/LingoMalay/Models_Transcribe/Dataset/extra/audio_2025-04-14_12-46-07.mp3"
# audio_path = "D:/LingoMalay/Models_Transcribe/Dataset/extra/audio_2025-04-19_20-46-14.mp3"
waveform, sample_rate = torchaudio.load(audio_path)

# Resample to 16kHz if needed
if sample_rate != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    waveform = resampler(waveform)
    sample_rate = 16000

# === Preprocess ===
input_data = processor(waveform.squeeze().numpy(), sampling_rate=16000, return_tensors="pt")
input_data["input_features"] = input_data["input_features"].cuda()

# === Build Prompt IDs ===
# Get Whisper's default prompt for Malay + transcribe
# prompt = processor.get_decoder_prompt_ids(language="ms", task="transcribe")

# Insert your dialect token (e.g. "<|kedah|>")
dialect_token = "<|kedah|>"
dialect_token_id = processor.tokenizer.convert_tokens_to_ids(dialect_token)

# Build custom prompt: e.g. [<|startoftranscript|>, <|kedah|>, <|ms|>, <|transcribe|>]
# custom_prompt_ids = prompt[:1] + [(1, dialect_token_id)] + prompt[1:]
# prompt_ids = [token_id for _, token_id in custom_prompt_ids]
# prompt_ids_tensor = torch.tensor([prompt_ids]).cuda()
# prompt_ids_tensor = torch.tensor([prompt_ids], dtype=torch.long).cuda()
# prompt_ids_tensor = torch.tensor(prompt_ids, dtype=torch.long).cuda()

# decoder_input_ids = torch.tensor([[token_id for _, token_id in prompt]], device="cuda")
# decoder_input_ids = torch.tensor([prompt_ids], dtype=torch.long).cuda()

prompt_tokens = tokenizer("<kedah>", add_special_tokens=False).input_ids
decoder_input_ids = torch.tensor([prompt_tokens]).cuda()

model.generation_config.forced_decoder_ids = None

# === Inference ===
with torch.no_grad():
    output = model.generate(
        input_data["input_features"],
        # prompt_ids=prompt_ids_tensor,
        decoder_input_ids=decoder_input_ids,
        return_dict_in_generate=True,
        return_timestamps=True,
        output_scores=True,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
        max_new_tokens=16,
        # do_sample=False,
        # num_beams=1,
    )

# === Decode ===
decoded = processor.batch_decode(output["sequences"], skip_special_tokens=True)[0]
print("🔹 Transcription:", decoded)

NameError: name 'tokenizer' is not defined

In [6]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import Audio

# Load processor and fine-tuned model
processor = WhisperProcessor.from_pretrained("mesolitica/malaysian-whisper-small-v3")
model = WhisperForConditionalGeneration.from_pretrained(r"D:\LingoMalay\Models_Transcribe\whisper-kedahv2\checkpoint-1000")

model.eval()

# Load and preprocess audio file
def load_audio(file_path):
    # Use datasets.Audio to load and resample the audio
    audio_loader = Audio(sampling_rate=16000)
    audio_data = audio_loader.decode_example(file_path)
    return audio_data["array"]

# Transcription function
def transcribe_audio(file_path):
    audio_array = load_audio(file_path)

    # Prepare input features
    input_features = processor.feature_extractor(audio_array, sampling_rate=16000).input_features
    input_features = torch.tensor(input_features).unsqueeze(0)  # batch dimension

    # Optionally, define the prompt with dialect token if needed
    # prompt = "<|startoftranscript|><|ms|><|kedah|><|transcribe|> "
    # processor.tokenizer.set_prompt(tokenizer(prompt).input_ids) # this is advanced, optional

    # Generate tokens
    with torch.no_grad():
        generated_ids = model.generate(input_features)

    # Decode to text
    transcription = processor.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return transcription

if __name__ == "__main__":
    file_path = r"D:/LingoMalay/Models_Transcribe/Dataset/extra/audio_2025-04-19_21-00-49.mp3"
    text = transcribe_audio(file_path)
    print("Transcription:", text)

TypeError: string indices must be integers

In [27]:
processor.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('D:\\LingoMalay\\Models_Transcribe\\whisper-kedah\\checkpoint-1000\\tokenizer_config.json',
 'D:\\LingoMalay\\Models_Transcribe\\whisper-kedah\\checkpoint-1000\\special_tokens_map.json',
 'D:\\LingoMalay\\Models_Transcribe\\whisper-kedah\\checkpoint-1000\\vocab.json',
 'D:\\LingoMalay\\Models_Transcribe\\whisper-kedah\\checkpoint-1000\\merges.txt',
 'D:\\LingoMalay\\Models_Transcribe\\whisper-kedah\\checkpoint-1000\\normalizer.json',
 'D:\\LingoMalay\\Models_Transcribe\\whisper-kedah\\checkpoint-1000\\added_tokens.json',
 'D:\\LingoMalay\\Models_Transcribe\\whisper-kedah\\checkpoint-1000\\tokenizer.json')

In [52]:
import torchaudio

def transcribe(audio_path, processor, model, tokenizer, dialect="<kedah>"):
    import torchaudio
    import torch

    # Load audio
    waveform, sample_rate = torchaudio.load(audio_path)
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)
    waveform = waveform.squeeze().numpy()

    # Prepare input features
    p = processor([waveform], return_tensors="pt", sampling_rate=16000)
    p = {k: v.to(model.device) for k, v in p.items()}

    # Optional: prompt with dialect token (e.g. <ms> <kedah>)
    prompt_ids = tokenizer("<ms> " + dialect, add_special_tokens=False).input_ids
    decoder_input_ids = torch.tensor([prompt_ids], device=model.device)

    # Generate with task and language
    with torch.no_grad():
        output = model.generate(
            p["input_features"],
            decoder_input_ids=decoder_input_ids,
            return_dict_in_generate=True,
            output_scores=True,
            language="ms",
            task="transcribe",
            return_timestamps=True,
        )

    # Decode result
    transcription = tokenizer.decode(output.sequences[0], skip_special_tokens=True)
    return transcription


from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperTokenizer
from datasets import Audio

model = WhisperForConditionalGeneration.from_pretrained(r"D:\LingoMalay\Models_Transcribe\whisper-kedah\checkpoint-1000").to("cuda" if torch.cuda.is_available() else "cpu")
model = WhisperForConditionalGeneration.from_pretrained("mesolitica/malaysian-whisper-small-v3").to("cuda" if torch.cuda.is_available() else "cpu")

processor = WhisperProcessor.from_pretrained("mesolitica/malaysian-whisper-small-v3")
tokenizer = WhisperTokenizer.from_pretrained("mesolitica/malaysian-whisper-small-v3")

tokenizer.add_special_tokens({'additional_special_tokens': ['<kedah>', '<kelantan>']})
model.resize_token_embeddings(len(tokenizer))
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.generation_config.forced_decoder_ids = None
model.generation_config._from_model_config = True

result = transcribe(r"D:\LingoMalay\Models_Transcribe\Dataset\extra\audio_2025-04-19_21-00-49.mp3", processor, model, tokenizer)
print(result)

The following generation flags are not valid and may be ignored: ['output_scores']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


AttributeError: 'dict' object has no attribute 'sequences'

In [47]:
import torch
from transformers.models.whisper import tokenization_whisper

tokenization_whisper.TASK_IDS = ["translate", "transcribe", "transcribeprecise"]

from transformers import WhisperForConditionalGeneration, WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    'malaysia-ai/malaysian-whisper-small'
)
tokenizer = processor.tokenizer
model = WhisperForConditionalGeneration.from_pretrained(
    'malaysia-ai/malaysian-whisper-small', torch_dtype = torch.bfloat16
).cuda().eval()


In [ ]:
from datasets import Audio
import requests

sr = 16000
audio = Audio(sampling_rate=sr)

r = requests.get('https://github.com/mesolitica/malaya-speech/raw/master/speech/assembly.mp3')
y = audio.decode_example(audio.encode_example(r.content))['array']

with torch.no_grad():
    p = processor([y], return_tensors='pt')
    p['input_features'] = p['input_features'].to(torch.bfloat16)
    r = model.generate(
        p['input_features'].cuda(),
        output_scores=True,
        return_dict_in_generate=True,
        language='ms',
        return_timestamps=True, task = 'transcribe')

tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(r.sequences[0]))

It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
The following generation flags are not valid and may be ignored: ['output_scores']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
You have passed `return_dict_in_generate=True` and `return_timestamps=True`, this automatically sets `return_segments=True` to access the results of the underlying calls to GenerationMixin's generate in the returned `segments`.


AttributeError: 'dict' object has no attribute 'sequences'

here2

In [7]:
from typing_extensions import TypeIs
from transformers import pipeline

# Point to the directory where the model is saved locally
# local_model_path = r"C:\Training\malaysian-whisper-large-v3-turbo-v3"
local_model_path = "openai/whisper-small"

asr = pipeline(
    "automatic-speech-recognition",
    model=local_model_path  # Use the local path instead of the Hugging Face model ID
    # tokenizer=local_model_path,
    # feature_extractor=local_model_path,
    # use_fast=False  # 🛠️ important!
)

print(asr.model)       # The actual model (e.g. WhisperForConditionalGeneration)
# print(asr.tokenizer)   # The tokenizer
# print(asr.feature_extractor)  # For audio input processing

# print(asr.model.config)
# for name, module in asr.model.named_modules():
#     print(name)

# result = model(r"C:\LingoMalay\assets\audio\audio1.wav")
# # model(audio_path, generate_kwargs={"language": "en"})
# print(result['text'])


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

Load Dataset

In [3]:
from datasets import load_dataset

# Load the Malaysian-STT-Whisper dataset
dataset = load_dataset("mesolitica/Malaysian-STT-Whisper")


In [4]:
# Check the structure
print(dataset)

# View a sample from the training data
# print(dataset["train"][0])

DatasetDict({
    extra: Dataset({
        features: ['audio_filename', 'segment_timestamp', 'word_timestamp'],
        num_rows: 7746
    })
    imda: Dataset({
        features: ['audio_filename', 'segment_timestamp', 'word_timestamp'],
        num_rows: 1009338
    })
    indonesian: Dataset({
        features: ['audio_filename', 'segment_timestamp', 'word_timestamp'],
        num_rows: 943282
    })
    malaysian_context: Dataset({
        features: ['audio_filename', 'segment_timestamp', 'word_timestamp'],
        num_rows: 299359
    })
    malaysian_context_v2: Dataset({
        features: ['audio_filename', 'segment_timestamp', 'word_timestamp'],
        num_rows: 3305115
    })
    mandarin: Dataset({
        features: ['audio_filename', 'segment_timestamp', 'word_timestamp'],
        num_rows: 1347342
    })
    science_context: Dataset({
        features: ['audio_filename', 'segment_timestamp', 'word_timestamp'],
        num_rows: 1960115
    })
    science_en: Dataset({
    

Evaluate

Retrain

In [ ]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration, WhisperConfig, Seq2SeqTrainer, Seq2SeqTrainingArguments

# Step 1: Load base config
config = WhisperConfig.from_pretrained("mesolitica/malaysian-whisper-large-v3-turbo-v3")
config.dropout = 0.1
config.attention_dropout = 0.1
config.apply_spec_augment = True

# Step 2: Load model with updated config
model = WhisperForConditionalGeneration(config)

# Step 3: Create processor & dataset
processor = WhisperProcessor.from_pretrained("mesolitica/malaysian-whisper-large-v3-turbo-v3")

# Step 4: Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-finetuned-ms",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    save_strategy="steps",
    learning_rate=1e-5,
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    num_train_epochs=3,
    fp16=True,
    predict_with_generate=True,
)

# Step 5: Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=processor.feature_extractor,
    data_collator=...,  # Use Whisper-specific collator
)

trainer.train()
